In [102]:
import nflgame
import pandas as pd
import numpy as np

# Get Data

In [167]:
# NFL Games for 2016
games = nflgame.games(2016)

In [55]:
players = nflgame.combine_game_stats(games)

In [166]:
# Get the Top 50 players who have the most rushing yards from the 2016 season
top50 = []
for p in players.rushing().sort('rushing_yds').limit(50):
    top50.append(p)

In [144]:
# Build DataFrame - Index for Player name and week #
top50_names = [str(p.name) for p in top50]
wks = range(1,18)
idxs = [top50_names, wks]

idx = pd.MultiIndex.from_product(idxs, names=['name', 'week'])
columns = ['rushing_att', 'rushing_yds', 'rushing_tds', 'fumbles']
df = pd.DataFrame(data=None, index=idx, columns=columns)

In [145]:
df.head()

rushing_att rushing_yds rushing_tds fumbles
name      week                                            
E.Elliott 1            NaN         NaN         NaN     NaN
          2            NaN         NaN         NaN     NaN
          3            NaN         NaN         NaN     NaN
          4            NaN         NaN         NaN     NaN
          5            NaN         NaN         NaN     NaN

In [163]:
# Update DF with stats for Top 50 players for all games
for g in games:
    wk = g.schedule['week']
    for p in g.players.rushing():
        
        # Check if top50 player
        if p.name in df.index:
            df.loc[(p.name, wk), 'rushing_att'] = p.rushing_att
            df.loc[(p.name, wk), 'rushing_yds'] = p.rushing_yds
            df.loc[(p.name, wk), 'rushing_tds'] = p.rushing_tds
            df.loc[(p.name, wk), 'fumbles_tot'] = p.fumbles_tot

In [169]:
# Calculate Fantasy Points
df['fantasy_points'] = df['rushing_yds'] * .1 + df['rushing_tds'] * 6 - df['fumbles_tot'] * 2

In [170]:
df

rushing_att rushing_yds rushing_tds fumbles  fumbles_tot  \
name      week                                                            
E.Elliott 1             20          51           1       0          0.0   
          2             21          83           1       2          2.0   
          3             30         140           0       0          0.0   
          4             23         138           1       0          0.0   
          5             15         134           2       0          0.0   
          6             28         157           0       0          0.0   
          7            NaN         NaN         NaN     NaN          NaN   
          8             22          96           0       0          0.0   
          9             18          92           2       1          1.0   
          10            21         114           2       0          0.0   
          11            25          97           0       0          0.0   
          12            20          97           2       0          0.0   
          13            20          86           1       1          1.0   
          14            24         107           0       0          0.0   
          15            23         159           1       1          1.0   
          16            12          80           2       0          0.0   
          17           NaN         NaN         NaN     NaN          NaN   
J.Howard  1            NaN         NaN         NaN     NaN          NaN   
          2              3          22           0       0          0.0   
          3              9          45           0       1          1.0   
          4             23         111           0       0          0.0   
          5             16         118           0       0          0.0   
          6             15          34           1       0          0.0   
          7              7          22           0       0          0.0   
          8             26         153           1       0          0.0   
          9            NaN         NaN         NaN     NaN          NaN   
          10            15         100           0       1          1.0   
          11            17          77           0       0          0.0   
          12            18          84           0       0          0.0   
          13            32         117           3       0          0.0   
...                    ...         ...         ...     ...          ...   
K.Dixon   5              3          -1           0       0          0.0   
          6              2          11           0       0          0.0   
          7              1           0           0       0          0.0   
          8            NaN         NaN         NaN     NaN          NaN   
          9              9          13           0       0          0.0   
          10             6          38           0       0          0.0   
          11             6          40           0       0          0.0   
          12            13          49           0       0          0.0   
          13             6          56           0       0          0.0   
          14            11          39           0       0          0.0   
          15             9          36           1       0          0.0   
          16            12          57           0       1          1.0   
          17            10          44           1       0          0.0   
A.Rodgers 1              4          16           1       0          0.0   
          2              3          29           1       3          3.0   
          3              5          22           0       0          0.0   
          4            NaN         NaN         NaN     NaN          NaN   
          5              5          15           0       0          0.0   
          6              3           7           0       2          2.0   
          7              2           7           0       1          1.0   
          8              6          60  

# Format Data

# Build Model

In [168]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

column_nums = 4
window_size = 1
model = Sequential()
model.add(LSTM(4, input_dim=column_nums*window_size))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

Using TensorFlow backend.
/Users/bvz/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
/Users/bvz/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(4, input_shape=(None, 4))`
  


In [ ]:
trainX = []
trainY = []
model.fit(trainX, trainY, nb_epoch=100, batch_size=3, verbose=2)

# Train Model